In [ ]:
%%capture
!pip install tenseal==0.3.0a2

In [ ]:
import syft as sy
sy.load_lib("tenseal")
duet = sy.launch_duet(loopback=True)

In [ ]:
import tenseal as ts

In [ ]:
# from: https://github.com/OpenMined/TenSEAL/blob/master/README.md

In [ ]:
# STEP 1 in TenSEAL
# Load Secret Vectors and Encrypt with CKKS Algorithm

In [ ]:
# TODO support property constructors like: ts.SCHEME_TYPE.CKKS
context = ts.Context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
# TODO: support remote @property.setter
context.global_scale = 2**40
context

In [ ]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

In [ ]:
enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)
(enc_v1, enc_v2)

In [ ]:
# STEP 2 Make Context and Encrypted Vectors Referenceable over Duet

In [ ]:
# tag them so our partner can easily reference it
ctx_ptr = context.send(duet, searchable=True, tags=["context"])
enc_v1_ptr = enc_v1.send(duet, searchable=True, tags=["enc_v1"])
enc_v2_ptr = enc_v2.send(duet, searchable=True, tags=["enc_v2"])

In [ ]:
# we can see that our three objects are now inside the store we control
duet.store.pandas

In [ ]:
# STOP and run the DS notebook

In [ ]:
# We can see our duet partner has requested the two encrypted vectors
duet.requests.pandas

In [ ]:
# STEP 3 Approve the two requests
duet.requests[0].accept()
duet.requests[0].accept()
duet.requests[0].accept()

In [ ]:
# The requests should have been handled
duet.requests.pandas

In [ ]:
# STOP and go run the DS notebook again

In [ ]:
# STEP 4 Get result_add from store and decrypt locally

In [ ]:
result_add_ptr = duet.store["result_add"]
result_add_ptr.link_context(ctx_ptr)

result_add = result_add_ptr.get(delete_obj=False)
result_add.link_context(context)

result_add

In [ ]:
decrypted_result = result_add.decrypt()
decrypted_result

In [ ]:
def _almost_equal(vec1, vec2, precision_pow_ten):
    if len(vec1) != len(vec2):
        return False
 
    upper_bound = pow(10, -precision_pow_ten)
    for v1, v2 in zip(vec1, vec2):
        if abs(v1 - v2) > upper_bound:
            return False
    return True

assert _almost_equal(decrypted_result, [4.0, 4.0, 4.0, 4.0, 4.0], 4)

In [ ]:
# TODO finish the other examples

In [ ]:
# duet.requests.add_handler(
#     name="decrypt",
#     action="deny",
#     print_local=True,
#     timeout_secs=100,
# )
# duet.requests.pandas

In [ ]:
# duet.requests

In [ ]:
# duet.requests.handlers

In [ ]:
# duet.requests.clear_handlers()

In [ ]:
duet.store.pandas